**LIBRERIAS**

In [2]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import LabelEncoder

**ABRIMOS LOS ARCHIVOS**

In [3]:
#Importamos los df
df_games = pd.read_csv('archivos definitivos/games.csv')
df_games_1 = pd.read_csv('games.csv')

*Despues de multiples pruebas de dsitintos tipo, llego a la conclusion que el quitar datos me esta quitando informacion que me es util, por ejemplo,PCA me quita informacion que es muy importante para el analisis*

*Despues de tantas prueba llegué a una logica*

In [1]:
#La consigna pide llegar a un MVP, quiere una lista de los 5 juegos mas recomendados en comparacion al item dado. La comparacion es de item a item, por logica los que más tienen relacion y serian más recomendados son aquellos juegos en donde su genero es igual. Entonces necesitaré otra columna que se encargue de hacer la similitud del coseno. Vamos a evaluar las etiquetas que tiene el juego, su jugabilidad, y demás. Todo esto lo podemos aprecia con 'tags'. Entonces este será el encargado hacer la comparación del angulo del coseno para determinar una simimlitud. 

*empezamos de cero*

In [ ]:
#extraemos el df_games_union para usarlo en la fastapi, este df lo unico que tiene de mas es tags
df_games_union.to_csv('games_union_ml.csv',index=False)

In [ ]:
#hacemos una union por el id para tener los tags
df_games_union = df_games_2.merge(df_games_complemento, left_on='item_id', right_index=True)

In [95]:
#se mermaron solo algunos pocas filas pero igual no es suficiente para hacer la muestra
df_games_union

,item_id,genres,year_release,tags
0,761140,Action,2018,Fantasy
1,761140,Casual,2018,Fantasy
2,761140,Indie,2018,Fantasy
3,761140,Simulation,2018,Fantasy
4,761140,Strategy,2018,Fantasy
...,...,...,...,...
71260,610660,Indie,2018,Indie
71261,610660,Racing,2018,Indie
71262,610660,Simulation,2018,Indie
71263,658870,Casual,2017,Shoot 'Em Up


*Empezamos a aplicar la codigo para armar la función*

In [53]:
#Filtra el DataFrame para obtener los juegos del género de referencia. Supongamos que deseas recomendar juegos similares al género "Action" y tienes el género de referencia en una variable llamada genre_reference:
genre_reference = "Action"
df_filtered = df_games_union[df_games_union['genres'] == genre_reference]

In [57]:
#Crea una matriz de características donde cada fila representa un juego y cada columna representa un género. Si un juego pertenece a un género, el valor en la columna correspondiente es 1; de lo contrario, es 0. Esto se hace utilizando la función pivot_table en pandas:
df_games_encoded = pd.get_dummies(df_filtered, columns=['tags'], dtype=int)

In [59]:
df_games_encoded.head()

,item_id,genres,year_release,tags_1980s,tags_1990's,tags_2.5D,tags_2D,tags_2D Fighter,tags_3D Platformer,tags_3D Vision,...,tags_Walking Simulator,tags_War,tags_Wargame,tags_Web Publishing,tags_Werewolves,tags_Word Game,tags_World War I,tags_World War II,tags_Zombies,tags_e-sports
0,761140,Action,2018,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
14,767400,Action,2017,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
17,772540,Action,2018,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
36,770380,Action,2018,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
64,768060,Action,2018,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [62]:
#eliminamos columnas insignificantes
df_games_encoded.drop(columns=['genres'], axis=1, inplace=True)

In [63]:
#realizamos la similitud del coseno
cosine_sim = cosine_similarity(df_games_encoded, df_games_encoded)

In [69]:
# Índice del juego de referencia
reference_game_index = df_filtered.index.values[0]

In [70]:
#corroboramos que lo que haya en la variable tenga sentido
reference_game_index

0

In [71]:
# Obtenemos las similitudes del juego de referencia con otros juegos
similarities = cosine_sim[reference_game_index]

In [73]:
# Enumeramos las similitudes junto con los índices de los juegos
similar_games = list(enumerate(similarities))

In [75]:
# Ordenamos la lista de juegos similares por similitud en orden descendente
similar_games_sorted = sorted(similar_games, key=lambda x: x[1], reverse=True)

In [77]:
# Excluimos el juego de referencia y toma los 5 juegos más similares
top_similar_games = similar_games_sorted[1:6]

In [79]:
# Obtenemos los índices de los juegos recomendados
recommended_game_indices = [game[0] for game in top_similar_games]

In [80]:
#verificamos
recommended_game_indices

[4653, 4372, 4370, 11200, 4354]

In [81]:
# guardamos en una variable(se guardara como df)
recommended_games = df_filtered.iloc[recommended_game_indices]

In [82]:
#verificamos la repuesta
recommended_games

,item_id,genres,year_release,tags
29827,760720,Action,2017,Casual
27875,760860,Action,2017,Adventure
27860,760900,Action,2017,Platformer
70965,761100,Action,2017,PvP
27762,752060,Action,1993,Simulation


**METEMOS EL MODELO EN LA FUNCION**

In [92]:
#definimos la funcion en donde le deben de pasar de parametro un id de un producto el cual quiero que tomen como referencia para mis 5 posteriores recomendaciones
def recomendacion_juego( id : int ):
    try:
        #Filtra el DataFrame para obtener los juegos del género de referencia. Lo guardamos en una variable llamada genre_reference:
        genre_reference = df_games_union.loc[df_games_union['item_id'] == id, 'genres'].iloc[0]
        df_filtered = df_games_union[df_games_union['genres'] == genre_reference]

        #Creo una matriz de características donde cada fila representa un juego y cada columna representa un tags. Si un juego pertenece a un tags, el valor en la columna correspondiente es 1; de lo contrario, es 0. Esto se hace utilizando la función pivot_table en pandas:
        df_games_encoded = pd.get_dummies(df_filtered, columns=['tags'], dtype=int)

        #eliminamos columnas insignificantes
        df_games_encoded.drop(columns=['genres'], axis=1, inplace=True)

        #esta es la parte pesada, tenemos que calcular la similitud del coseno
        cosine_sim = cosine_similarity(df_games_encoded, df_games_encoded)

        # guardamos el indice del juego de referencia
        reference_game_index = df_filtered.index.values[0]

        # Obtenemos las similitudes del juego de referencia con otros juegos
        similarities = cosine_sim[reference_game_index]

        # Enumeramos las similitudes junto con los índices de los juegos
        similar_games = list(enumerate(similarities))

        # Ordenamos la lista de juegos similares por similitud en orden descendente
        similar_games_sorted = sorted(similar_games, key=lambda x: x[1], reverse=True)

        # Excluhimos el juego de referencia y toma los 5 juegos más similares
        top_similar_games = similar_games_sorted[1:6]

        # Obtenemos los índices de los juegos recomendados
        recommended_game_indices = [game[0] for game in top_similar_games]

        return {
            'Top 5 de id de los juegos más recomendados similares al que ingreso' : f'{recommended_game_indices}'
        }
    except Exception:
        return 'Por favor ingrese un id valido'

In [94]:
recomendacion_juego(10)

{'Top 5 de id de los juegos más recomendados similares al que ingreso': '[4653, 4372, 4370, 11200, 4354]'}